<a href="https://colab.research.google.com/github/szha0052/Final-5329A1/blob/main/Grid_search%26Ablation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/COMP5329_A1/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/COMP5329_A1


In [14]:
import numpy as np
from MLP.model import MLP
import pandas as pd
from MLP.layers import SoftmaxCrossEntropy
from MLP.layers import Adam
from MLP.layers import MiniBatchFit
from MLP.layers import SGDMomentum
from tqdm import tqdm



Mounted at /content/drive/
/content/drive/MyDrive/COMP5329_A1


## 1. Data Import and Data Preprocessing

## 1.1 Import Data

In [15]:
# import npy data
X_train = np.load('Assignment1-Dataset/train_data.npy')
X_test = np.load('Assignment1-Dataset/test_data.npy')
Y_train = np.load('Assignment1-Dataset/train_label.npy').flatten()
Y_test = np.load('Assignment1-Dataset/test_label.npy').flatten()

In [16]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((50000, 128), (50000,), (10000, 128), (10000,))

In [17]:
# Statistical data categories
classes = np.unique(Y_train)
classes

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

### 1.2 Data Preprocessing

In [ ]:
def stratified_train_test_split(X, y, test_size=0.2, random_seed=None):
    """
    Stratified sampling-based train-test split implemented using NumPy.

    :param X: Feature array, shape (N, d), where N is the number of samples and d is the feature dimension
    :param y: Label array, shape (N, )
    :param test_size: Proportion of the test set (0.0 ~ 1.0)
    :param random_seed: Random seed for reproducibility
    :return: (X_train, X_test, y_train, y_test)
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    # Find all unique classes in the labels and their corresponding indices
    unique_labels = np.unique(y)

    train_indices = []
    test_indices = []

    for label in unique_labels:
        # Find all indices of samples belonging to the current class
        label_indices = np.where(y == label)[0]

        # Shuffle the indices of the current class to avoid order bias
        np.random.shuffle(label_indices)

        # Calculate the number of samples to allocate to the test set
        test_count = int(len(label_indices) * test_size)

        # Assign the first test_count samples to the test set, and the rest to the train set
        test_indices.append(label_indices[:test_count])
        train_indices.append(label_indices[test_count:])

    # Concatenate indices from all classes
    train_indices = np.concatenate(train_indices)
    test_indices = np.concatenate(test_indices)

    # Optionally shuffle the train and test indices further
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)

    # Split the data based on the indices
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    return X_train, X_test, y_train, y_test

In [ ]:
def classification_report_numpy(y_true, y_pred, average='macro'):
    """
    Compute multi-class Precision, Recall, and F1 using only NumPy.

    :param y_true: True labels, shape (N,)
    :param y_pred: Predicted labels, shape (N,)
    :param average: Specify averaging method, options are 'macro' or 'micro'
    :return: (precision, recall, f1)
             - If average='macro', returns macro-averaged (P, R, F1)
             - If average='micro', returns micro-averaged (P, R, F1)
    """
    # Collect all classes and create an index mapping (if needed)
    classes = np.unique(np.concatenate((y_true, y_pred)))
    num_classes = len(classes)

    # Build confusion matrix
    conf_mat = np.zeros((num_classes, num_classes), dtype=np.int64)

    # Create a mapping dictionary from label to index
    label_to_index = {label: idx for idx, label in enumerate(classes)}

    for t, p in zip(y_true, y_pred):
        i = label_to_index[t]
        j = label_to_index[p]
        conf_mat[i, j] += 1

    if average == 'macro':
        # Macro average: compute for each class and then average
        precisions = []
        recalls = []
        f1s = []
        for i in range(num_classes):
            TP = conf_mat[i, i]
            FP = conf_mat[:, i].sum() - TP
            FN = conf_mat[i, :].sum() - TP

            prec = TP / (TP + FP) if (TP + FP) != 0 else 0.0
            rec = TP / (TP + FN) if (TP + FN) != 0 else 0.0
            f1 = 2 * prec * rec / (prec + rec) if (prec + rec) != 0 else 0.0

            precisions.append(prec)
            recalls.append(rec)
            f1s.append(f1)

        precision_macro = np.mean(precisions)
        recall_macro = np.mean(recalls)
        f1_macro = np.mean(f1s)
        return precision_macro, recall_macro, f1_macro

    elif average == 'micro':
        # Micro average: compute based on global TP, FP, FN
        TP_total = np.diag(conf_mat).sum()
        # For each column: column sum - diagonal element, i.e., all false positives in the column
        FP_total = conf_mat.sum(axis=0) - np.diag(conf_mat)
        # For each row: row sum - diagonal element, i.e., all false negatives in the row
        FN_total = conf_mat.sum(axis=1) - np.diag(conf_mat)

        FP_total = FP_total.sum()
        FN_total = FN_total.sum()

        precision_micro = TP_total / (TP_total + FP_total) if (TP_total + FP_total) != 0 else 0.0
        recall_micro = TP_total / (TP_total + FN_total) if (TP_total + FN_total) != 0 else 0.0
        f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) \
            if (precision_micro + recall_micro) != 0 else 0.0

        return precision_micro, recall_micro, f1_micro

    else:
        raise ValueError("Unsupported average type. Use 'macro' or 'micro'.")

In [ ]:
# Stratified train-test split
X_train, X_val, Y_train, Y_val = stratified_train_test_split(X_train, Y_train, test_size=0.2, random_seed=42)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((40000, 128), (40000,), (10000, 128), (10000,))

In [ ]:
# Check the distribution of classes in the training and val sets
pd.Series(Y_train).value_counts(), pd.Series(Y_test).value_counts()


(6    4000
 7    4000
 8    4000
 3    4000
 2    4000
 1    4000
 4    4000
 9    4000
 0    4000
 5    4000
 Name: count, dtype: int64,
 3    1000
 8    1000
 0    1000
 6    1000
 1    1000
 9    1000
 5    1000
 7    1000
 4    1000
 2    1000
 Name: count, dtype: int64)

## 2. Grid Search

In [ ]:
input_dim = 128
hidden_dims = [64, 32]
output_dim = 10
# learning_rate = 0.001
momentum = 0.9
num_epochs = 200
batch_size = 100


### 2.1 Adam Optimizer for Grid Search

In [ ]:

# Define hyperparameter search space for Adam
param_grid = {
    'learning_rate': [0.001, 0.005, 0.01],
    'weight_decay': [0.0, 1e-5, 1e-4],
    'dropout_rate': [0.1, 0.2, 0.3]
}

# Calculate total combinations for Adam
total_combinations_adam = len(param_grid['learning_rate']) * len(param_grid['weight_decay']) * len(param_grid['dropout_rate'])
# Initialize tqdm progress bar for Adam
adam_tqdm = tqdm(total=total_combinations_adam, desc="Adam Hyperparameter Search", unit="combination")

adam_search_results = []

# Manually iterate over all combinations (no itertools)
for lr in param_grid['learning_rate']:
    for wd in param_grid['weight_decay']:
        for drop in param_grid['dropout_rate']:
            optimizer = Adam()
            model = MLP(input_dim, hidden_dims, output_dim, dropout_rate=drop, weight_decay=wd, optimizer=optimizer)
            trainer = MiniBatchFit(model, optimizer, X_train, Y_train,
                                   output_dim, num_epochs=num_epochs,
                                   batch_size=batch_size,
                                   learning_rate=lr, write = False)
            trainer.fit()
            _, accuracy = model.predict_and_evaluate(X_test, Y_test)
            adam_search_results.append({
                'learning_rate': lr,
                'weight_decay': wd,
                'dropout_rate': drop,
                'test_accuracy': round(accuracy, 4)
            })
            # Update progress bar
            adam_tqdm.update(1)

# Close the progress bar
adam_tqdm.close()

# Save results to DataFrame
adam_result_df = pd.DataFrame(adam_search_results)
adam_result_df = adam_result_df.sort_values(by='test_accuracy', ascending=False).reset_index(drop=True)

# Show top 5 configurations
print("Top 5 Adam Hyperparameter Configurations:")
print(adam_result_df.head())

Adam Hyperparameter Search: 100%|██████████| 27/27 [35:23<00:00, 78.65s/combination]

Top 5 Adam Hyperparameter Configurations:
   learning_rate  weight_decay  dropout_rate  test_accuracy
0          0.001       0.00000           0.1         0.5215
1          0.010       0.00001           0.1         0.5196
2          0.005       0.00000           0.1         0.5183
3          0.001       0.00010           0.1         0.5179
4          0.001       0.00001           0.1         0.5176


#### 2.1.1 Optimal model training based on the Adam optimizer

In [ ]:

best_cfg = adam_result_df.iloc[0]
best_lr = best_cfg['learning_rate']
best_wd = best_cfg['weight_decay']
best_drop = best_cfg['dropout_rate']

#
optimizer1 = Adam()
model_Adam = MLP(input_dim, hidden_dims, output_dim,
                 dropout_rate=best_drop,
                 weight_decay=best_wd,
                 optimizer=optimizer1)
Adam_Fit = MiniBatchFit(model_Adam, optimizer1,
             X_train, Y_train,
             output_dim,
             num_epochs=num_epochs,
             batch_size=batch_size,
             learning_rate=best_lr)
Adam_Fit.fit()


Epoch 1/200, Loss: 1.9661
Epoch 2/200, Loss: 1.7698
Epoch 3/200, Loss: 1.7163
Epoch 4/200, Loss: 1.6858
Epoch 5/200, Loss: 1.6609
Epoch 6/200, Loss: 1.6450
Epoch 7/200, Loss: 1.6300
Epoch 8/200, Loss: 1.6173
Epoch 9/200, Loss: 1.6038
Epoch 10/200, Loss: 1.5935
Epoch 11/200, Loss: 1.5807
Epoch 12/200, Loss: 1.5743
Epoch 13/200, Loss: 1.5646
Epoch 14/200, Loss: 1.5591
Epoch 15/200, Loss: 1.5473
Epoch 16/200, Loss: 1.5427
Epoch 17/200, Loss: 1.5344
Epoch 18/200, Loss: 1.5257
Epoch 19/200, Loss: 1.5214
Epoch 20/200, Loss: 1.5139
Epoch 21/200, Loss: 1.5098
Epoch 22/200, Loss: 1.5024
Epoch 23/200, Loss: 1.4974
Epoch 24/200, Loss: 1.4906
Epoch 25/200, Loss: 1.4839
Epoch 26/200, Loss: 1.4787
Epoch 27/200, Loss: 1.4745
Epoch 28/200, Loss: 1.4705
Epoch 29/200, Loss: 1.4705
Epoch 30/200, Loss: 1.4616
Epoch 31/200, Loss: 1.4606
Epoch 32/200, Loss: 1.4537
Epoch 33/200, Loss: 1.4484
Epoch 34/200, Loss: 1.4479
Epoch 35/200, Loss: 1.4445
Epoch 36/200, Loss: 1.4402
Epoch 37/200, Loss: 1.4391
Epoch 38/2

#### 2.1.2 Evaluation of Validation Set

In [ ]:
predictions_train_Adam, Adam_Train_acc = model_Adam.predict_and_evaluate(X_val, Y_val)
precision_val,recall_val,f1_val = classification_report_numpy(Y_val, predictions_train_Adam, average='macro')
print(f"Validation Accuracy: {Adam_Train_acc:.4f}")
print(f"Validation F1 Score: {f1_val:.4f}")
print(f"Validation Recall: {recall_val:.4f}")
print(f"Validation Precision: {precision_val:.4f}")

Validation Accuracy: 0.5188
Validation F1 Score: 0.5114
Validation Recall: 0.5188
Validation Precision: 0.5153


#### 2.1.3 Evaluation of Test Set

In [ ]:
predictions_test_Adam,Adam_Test_acc = model_Adam.predict_and_evaluate(X_test, Y_test)
precision_test,recall_test,f1_test = classification_report_numpy(Y_test, predictions_test_Adam, average='macro')
print(f"Test Accuracy: {Adam_Test_acc:.4f}")
print(f"Test F1 Score: {f1_test:.4f}")
print(f"Test Recall: {recall_test:.4f}")
print(f"Test Precision: {precision_test:.4f}")

Test Accuracy: 0.5142
Test F1 Score: 0.5057
Test Recall: 0.5142
Test Precision: 0.5096


### 2.2 SGD Optimizer for Grid Search

In [ ]:
SGDM_param_grid = {
    'learning_rate': [0.001, 0.005, 0.01],
    'momentum': [0.5, 0.9],
    'dropout_rate': [0.1, 0.2, 0.3],
    'weight_decay': [0.0, 1e-5, 1e-4]
}

# Calculate total combinations for SGDM
total_combinations_sgdm = len(SGDM_param_grid['learning_rate']) * len(SGDM_param_grid['momentum']) * \
    len(SGDM_param_grid['dropout_rate']) * len(SGDM_param_grid['weight_decay'])
# Initialize tqdm progress bar for SGDM
sgdm_tqdm = tqdm(total=total_combinations_sgdm, desc="SGDM Hyperparameter Search", unit="combination")

SGDM_search_results = []

for lr in SGDM_param_grid['learning_rate']:
    for mom in SGDM_param_grid['momentum']:
        for drop in SGDM_param_grid['dropout_rate']:
            for wd in SGDM_param_grid['weight_decay']:
                optimizer = SGDMomentum(momentum=mom)
                model = MLP(input_dim, hidden_dims, output_dim, dropout_rate=drop, weight_decay=wd, optimizer=optimizer)
                trainer = MiniBatchFit(model, optimizer, X_train, Y_train,
                                       output_dim, num_epochs=num_epochs,
                                       batch_size=batch_size,
                                       learning_rate=lr, write = False)
                trainer.fit()
                _, accuracy = model.predict_and_evaluate(X_test, Y_test)
                SGDM_search_results.append({
                    'learning_rate': lr,
                    'momentum': mom,
                    'dropout_rate': drop,
                    'weight_decay': wd,
                    'test_accuracy': round(accuracy, 4)
                })
                # Update progress bar
                sgdm_tqdm.update(1)

# Close the progress bar
sgdm_tqdm.close()

# Save results to DataFrame
SGDM_result_df = pd.DataFrame(SGDM_search_results)
SGDM_result_df = SGDM_result_df.sort_values(by='test_accuracy', ascending=False).reset_index(drop=True)

# Save best SGDM hyperparameters
best_SGDM_cfg = SGDM_result_df.iloc[0]
best_SGDM_lr = best_SGDM_cfg['learning_rate']
best_SGDM_mom = best_SGDM_cfg['momentum']
best_SGDM_wd = best_SGDM_cfg['weight_decay']
best_SGDM_drop = best_SGDM_cfg['dropout_rate']

print("Top 5 SGDM Hyperparameter Configurations:")
print(SGDM_result_df.head())

SGDM Hyperparameter Search: 100%|██████████| 54/54 [1:01:24<00:00, 68.23s/combination]

Top 5 SGDM Hyperparameter Configurations:
   learning_rate  momentum  dropout_rate  weight_decay  test_accuracy
0          0.010       0.5           0.1       0.00000         0.5233
1          0.005       0.9           0.1       0.00010         0.5229
2          0.010       0.5           0.1       0.00010         0.5192
3          0.005       0.9           0.1       0.00001         0.5183
4          0.005       0.5           0.1       0.00001         0.5178


### 2.2.1 Optimal model training based on the SGD optimizer

In [ ]:
# === SGDM: Final Model Training with Best Hyperparameters ===

optimizer_SGDM = SGDMomentum(momentum=best_SGDM_mom)

model_SGDM = MLP(input_dim, hidden_dims, output_dim,
                 dropout_rate=best_SGDM_drop,
                 weight_decay=best_SGDM_wd,
                 optimizer=optimizer_SGDM)

SGDM_Fit = MiniBatchFit(model_SGDM, optimizer_SGDM,
                        X_train, Y_train,
                        output_dim=output_dim,
                        num_epochs=num_epochs,
                        batch_size=batch_size,
                        learning_rate=best_SGDM_lr)

SGDM_Fit.fit()


Epoch 1/200, Loss: 2.1423
Epoch 2/200, Loss: 1.9238
Epoch 3/200, Loss: 1.8435
Epoch 4/200, Loss: 1.7982
Epoch 5/200, Loss: 1.7582
Epoch 6/200, Loss: 1.7291
Epoch 7/200, Loss: 1.7044
Epoch 8/200, Loss: 1.6820
Epoch 9/200, Loss: 1.6624
Epoch 10/200, Loss: 1.6473
Epoch 11/200, Loss: 1.6344
Epoch 12/200, Loss: 1.6186
Epoch 13/200, Loss: 1.6066
Epoch 14/200, Loss: 1.5967
Epoch 15/200, Loss: 1.5873
Epoch 16/200, Loss: 1.5797
Epoch 17/200, Loss: 1.5715
Epoch 18/200, Loss: 1.5654
Epoch 19/200, Loss: 1.5551
Epoch 20/200, Loss: 1.5463
Epoch 21/200, Loss: 1.5415
Epoch 22/200, Loss: 1.5344
Epoch 23/200, Loss: 1.5269
Epoch 24/200, Loss: 1.5238
Epoch 25/200, Loss: 1.5155
Epoch 26/200, Loss: 1.5081
Epoch 27/200, Loss: 1.5026
Epoch 28/200, Loss: 1.4984
Epoch 29/200, Loss: 1.4933
Epoch 30/200, Loss: 1.4864
Epoch 31/200, Loss: 1.4811
Epoch 32/200, Loss: 1.4778
Epoch 33/200, Loss: 1.4754
Epoch 34/200, Loss: 1.4674
Epoch 35/200, Loss: 1.4664
Epoch 36/200, Loss: 1.4636
Epoch 37/200, Loss: 1.4561
Epoch 38/2

### 2.2.2 Evaluation of Validation Set

In [ ]:
# Get predictions and ignore accuracy here
y_pred, SGD_Train_acc = model_SGDM.predict_and_evaluate(X_val, Y_val)

# Evaluate metrics
precision_val, recall_val, f1_val = classification_report_numpy(Y_val, y_pred, average='macro')
print(f"Validation Accuracy: {SGD_Train_acc:.4f}")
print(f"Validation F1 Score: {f1_val:.4f}")
print(f"Validation Recall: {recall_val:.4f}")
print(f"Validation Precision: {precision_val:.4f}")


Validation Accuracy: 0.5196
Validation F1 Score: 0.5127
Validation Recall: 0.5196
Validation Precision: 0.5139


### 2.2.3 Evaluation of Test Set

In [ ]:
y_pred_test, SGD_Test_acc = model_SGDM.predict_and_evaluate(X_test, Y_test)

precision_test, recall_test, f1_test = classification_report_numpy(Y_test, y_pred_test, average='macro')
print(f"Test Accuracy: {SGD_Test_acc:.4f}")
print(f"Test F1 Score: {f1_test:.4f}")
print(f"Test Recall: {recall_test:.4f}")
print(f"Test Precision: {precision_test:.4f}")


Test Accuracy: 0.5213
Test F1 Score: 0.5142
Test Recall: 0.5213
Test Precision: 0.5153


## 3. Ablation experiment

### 3.1 Adam Optimizer without Dropout

In [ ]:
# adam ==0
# dropout == 0
optimizer1 = Adam()
model_Adam = MLP(input_dim, hidden_dims, output_dim,
                 dropout_rate=0,
                 weight_decay=0,
                 optimizer=optimizer1)
Adam_Fit = MiniBatchFit(model_Adam, optimizer1,
             X_train, Y_train,
             output_dim,
             num_epochs=num_epochs,
             batch_size=batch_size,
             learning_rate=0.001)
Adam_Fit.fit()


Epoch 1/200, Loss: 1.8795
Epoch 2/200, Loss: 1.6722
Epoch 3/200, Loss: 1.6160
Epoch 4/200, Loss: 1.5817
Epoch 5/200, Loss: 1.5560
Epoch 6/200, Loss: 1.5353
Epoch 7/200, Loss: 1.5175
Epoch 8/200, Loss: 1.5018
Epoch 9/200, Loss: 1.4877
Epoch 10/200, Loss: 1.4747
Epoch 11/200, Loss: 1.4626
Epoch 12/200, Loss: 1.4513
Epoch 13/200, Loss: 1.4405
Epoch 14/200, Loss: 1.4303
Epoch 15/200, Loss: 1.4204
Epoch 16/200, Loss: 1.4109
Epoch 17/200, Loss: 1.4018
Epoch 18/200, Loss: 1.3930
Epoch 19/200, Loss: 1.3845
Epoch 20/200, Loss: 1.3764
Epoch 21/200, Loss: 1.3687
Epoch 22/200, Loss: 1.3612
Epoch 23/200, Loss: 1.3541
Epoch 24/200, Loss: 1.3472
Epoch 25/200, Loss: 1.3406
Epoch 26/200, Loss: 1.3341
Epoch 27/200, Loss: 1.3279
Epoch 28/200, Loss: 1.3220
Epoch 29/200, Loss: 1.3162
Epoch 30/200, Loss: 1.3106
Epoch 31/200, Loss: 1.3051
Epoch 32/200, Loss: 1.2999
Epoch 33/200, Loss: 1.2947
Epoch 34/200, Loss: 1.2898
Epoch 35/200, Loss: 1.2850
Epoch 36/200, Loss: 1.2803
Epoch 37/200, Loss: 1.2758
Epoch 38/2

### 3.1.1 Evaluation of Validation Set

In [ ]:
predictions_train_Adam, Adam_Train_no_dropout_acc = model_Adam.predict_and_evaluate(X_val, Y_val)
Adam_Train_no_dropout_precision_val,Adam_Train_no_dropout_recall_val,Adam_Train_no_dropout_f1_val = classification_report_numpy(Y_val, predictions_train_Adam, average='macro')
print(f"Validation Accuracy: {Adam_Train_no_dropout_acc:.4f}")
print(f"Validation F1 Score: {Adam_Train_no_dropout_f1_val:.4f}")
print(f"Validation Recall: {Adam_Train_no_dropout_recall_val:.4f}")
print(f"Validation Precision: {Adam_Train_no_dropout_precision_val:.4f}")


Validation Accuracy: 0.4973
Validation F1 Score: 0.4939
Validation Recall: 0.4973
Validation Precision: 0.4923


### 3.1.2 Evaluation of Test Set

In [ ]:
predictions_test_Adam,Adam_Test_no_dropout_acc = model_Adam.predict_and_evaluate(X_test, Y_test)
Adam_Test_no_dropout_precision_test,Adam_Test_no_dropout_recall_test,Adam_Test_no_dropout_f1_test = classification_report_numpy(Y_test, predictions_test_Adam, average='macro')
print(f"Test Accuracy: {Adam_Test_acc:.4f}")
print(f"Test F1 Score: {Adam_Test_no_dropout_f1_test:.4f}")
print(f"Test Recall: {Adam_Test_no_dropout_recall_test:.4f}")
print(f"Test Precision: {Adam_Test_no_dropout_precision_test:.4f}")

Test Accuracy: 0.4855
Test F1 Score: 0.4828
Test Recall: 0.4868
Test Precision: 0.4814


## 3.2 SGD Optimizer without Dropout

In [ ]:
# SGD
# dropout == 0
optimizer_SGDM = SGDMomentum(momentum=0.5)

model_SGDM = MLP(input_dim, hidden_dims, output_dim,
                 dropout_rate=0.0,
                 weight_decay=0.0,
                 optimizer=optimizer_SGDM)

SGDM_Fit = MiniBatchFit(model_SGDM, optimizer_SGDM,
                        X_train, Y_train,
                        output_dim=output_dim,
                        num_epochs=num_epochs,
                        batch_size=batch_size,
                        learning_rate=0.01)

SGDM_Fit.fit()

Epoch 1/200, Loss: 2.0795
Epoch 2/200, Loss: 1.8262
Epoch 3/200, Loss: 1.7360
Epoch 4/200, Loss: 1.6808
Epoch 5/200, Loss: 1.6388
Epoch 6/200, Loss: 1.6043
Epoch 7/200, Loss: 1.5756
Epoch 8/200, Loss: 1.5513
Epoch 9/200, Loss: 1.5303
Epoch 10/200, Loss: 1.5118
Epoch 11/200, Loss: 1.4952
Epoch 12/200, Loss: 1.4800
Epoch 13/200, Loss: 1.4661
Epoch 14/200, Loss: 1.4531
Epoch 15/200, Loss: 1.4408
Epoch 16/200, Loss: 1.4294
Epoch 17/200, Loss: 1.4185
Epoch 18/200, Loss: 1.4079
Epoch 19/200, Loss: 1.3979
Epoch 20/200, Loss: 1.3881
Epoch 21/200, Loss: 1.3790
Epoch 22/200, Loss: 1.3703
Epoch 23/200, Loss: 1.3623
Epoch 24/200, Loss: 1.3548
Epoch 25/200, Loss: 1.3475
Epoch 26/200, Loss: 1.3406
Epoch 27/200, Loss: 1.3340
Epoch 28/200, Loss: 1.3277
Epoch 29/200, Loss: 1.3217
Epoch 30/200, Loss: 1.3159
Epoch 31/200, Loss: 1.3105
Epoch 32/200, Loss: 1.3052
Epoch 33/200, Loss: 1.3001
Epoch 34/200, Loss: 1.2952
Epoch 35/200, Loss: 1.2905
Epoch 36/200, Loss: 1.2857
Epoch 37/200, Loss: 1.2811
Epoch 38/2

### 3.2.1 Evaluation of Validation Set

In [ ]:
# Get predictions and ignore accuracy here
y_pred, SGD_Train_no_dropout_acc = model_SGDM.predict_and_evaluate(X_val, Y_val)

# Evaluate metrics
SGD_Train_no_dropout_precision_val, SGD_Train_no_dropout_recall_val, SGD_Train_no_dropout_f1_val = classification_report_numpy(Y_val, y_pred, average='macro')
print(f"Validation Accuracy: {SGD_Train_no_dropout_acc:.4f}")

print(f"Validation F1 Score: {SGD_Train_no_dropout_f1_val:.4f}")
print(f"Validation Recall: {SGD_Train_no_dropout_recall_val:.4f}")
print(f"Validation Precision: {SGD_Train_no_dropout_precision_val:.4f}")

Validation Accuracy: 0.5030
Validation F1 Score: 0.4976
Validation Recall: 0.5030
Validation Precision: 0.4958


### 3.2.2 Evaluation of Test Set

In [ ]:
y_pred_test, SGD_Test_no_dropout_acc = model_SGDM.predict_and_evaluate(X_test, Y_test)

SGD_Test_no_dropout_precision_test, SGD_Test_no_dropout_recall_test, SGD_Test_no_dropout_f1_test = classification_report_numpy(Y_test, y_pred_test, average='macro')
print(f"Test Accuracy: {SGD_Test_no_dropout_acc:.4f}")

print(f"Test F1 Score: {SGD_Test_no_dropout_f1_test:.4f}")
print(f"Test Recall: {SGD_Test_no_dropout_recall_test:.4f}")
print(f"Test Precision: {SGD_Test_no_dropout_precision_test:.4f}")

Test Accuracy: 0.4986
Test F1 Score: 0.4943
Test Recall: 0.4986
Test Precision: 0.4931
